In [ ]:
import scanpy as sc
import pandas as pd
import yaml
from sklearn.neighbors import KNeighborsClassifier 

In [ ]:
with open("config_dataset.yaml", "r") as stream:
    dataset_config = yaml.safe_load(stream)
samples = [*dataset_config["SAMPLE"], *dataset_config["SAMPLE_LQ"]]
samples

In [ ]:
for sample in samples:
    org_sample_name = sample.replace('-', '_')
    mtx_in_file = f"data/o30773_SpaceRangerCount_v2_1_0_2023-05-18--18-23-28/{org_sample_name}/filtered_feature_bc_matrix/"
    annotation_path = f"data/aestetik_supervised_clusters/aestetik_{org_sample_name}_supervised_1.5_bgm.csv"
    
    refined_cluster = pd.read_csv(annotation_path)
    refined_cluster = refined_cluster.set_index("Barcode")

    adata = sc.read_10x_mtx(mtx_in_file)
    break
    adata.obs = adata.obs.merge(refined_cluster, left_index=True, right_index=True, how="left")
    adata = adata[adata.obs["manual_anno"] != "EXCL",:] # remove empty spots
    adata = adata[~adata.obs["manual_anno"].isna(),:] # remove empty spots
    
    sc.pp.normalize_total(adata, target_sum=10000)
    sc.pp.log1p(adata)
    sc.pp.pca(adata)
    sc.pp.neighbors(adata)
    sc.tl.umap(adata)
    X = adata[adata.obs.aestetik_manual_anno != 'UNASSIGNED'].obsm["X_pca"]
    y = adata[adata.obs.aestetik_manual_anno != 'UNASSIGNED'].obs.aestetik_manual_anno
    knn = KNeighborsClassifier(5).fit(X, y)
    adata.obs["knn_label"] = knn.predict(adata.obsm["X_pca"])
    sc.pl.umap(adata, color=['aestetik_manual_anno', 'knn_label'])
    adata.obs[['manual_anno', 'aestetik_manual_anno', 'aestetik_manual_anno_tls', 'knn_label']].to_csv(f"data/knn_labels/{sample}.csv")

In [ ]:
adata.obs = adata.obs.merge(refined_cluster, left_index=True, right_index=True, how="left")
adata.obs